In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup 
import nltk
import warnings
import sklearn
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

import keras
import tensorflow as tf
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras import optimizers
tf.keras.layers.Embedding
#gensim w2v
#word2vec
from gensim.models import Word2Vec


In [2]:
data= pd.read_csv('cleaned1.csv')
data.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned1.csv'

In [218]:
data=data.drop(columns='Unnamed: 0',inplace=False)

In [219]:
df= data['review_body'].apply(str)

In [220]:
data['review_body']=df.apply(lambda x: re.sub('[^a-zA-Z ]', '', x))
data['review_body']=df.apply(lambda x: x.lower())


In [221]:
data

,review_body,star_rating,sentiment
0,provides great cushion as well as archsupport,5,1
1,it's perfect if you need something small for c...,4,1
2,getting what u see,5,1
3,small,3,1
4,my 13 year old son loved these shoes excellent...,5,1
...,...,...,...
99995,shoes began to &#34;peel&#34; within a month. ...,1,0
99996,i had ordered this bag in the herbal mist colo...,1,0
99997,great shoe but after wearing them for less the...,2,0
99998,it's a pretty shoe but runs small,2,0


In [222]:
# function to clean and pre-process the text.
def clean_reviews(review):  
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review,"lxml").get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    #le=WordNetLemmatizer()
    #stop_words= set(stopwords.words("english"))     
    #word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

# NLTK LIBRARY FOR CREATING TOKENIZING REVIEWS

In [223]:
#!pip install nltk
#nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0
for review_body in data['review_body']:
    sents=tokenizer.tokenize(review_body.strip())
    sum+=len(sents)
    for sent in sents:
        cleaned_sent=clean_reviews(sent)
        sentences.append(cleaned_sent.split()) # can use word_tokenize also.
print(sum)
print(len(sentences))  # total no of sentences

280197
280197


In [224]:
import gensim

w2v_model=gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=10,min_count=1)

In [225]:
w2v_model.train(sentences,epochs=10,total_examples=len(sentences))

(23800398, 33875000)

In [226]:
# embedding of a particular word.
w2v_model.wv.get_vector('great')

array([ 0.01482964,  0.05365279,  0.21364582,  1.509128  ,  0.19120501,
       -0.45705885,  0.69639635,  0.12131977, -0.1970114 , -1.5681069 ,
       -1.3750471 , -0.09908123, -0.18555643,  1.078124  , -1.8970355 ,
       -0.37271693,  0.3267752 ,  2.32171   , -0.04179703, -1.3103684 ,
        0.2839957 , -0.80827445, -0.6977456 , -0.24594198, -0.01212678,
       -0.68564117,  0.6389927 , -0.4106011 ,  0.0801646 , -0.65511036,
       -0.19347282,  1.1372072 , -1.1358166 , -0.9680011 ,  1.9426973 ,
        2.0871346 , -0.6104485 ,  0.7143133 , -1.5073532 ,  1.4545008 ,
        1.9574941 ,  0.00508789, -1.7841923 , -0.94573987,  0.32479912,
        0.0405994 ,  1.5777903 ,  1.2236146 ,  0.23646735,  1.3863181 ,
        0.546704  ,  1.1135038 , -0.50789195,  0.57979393, -0.15880561,
       -0.6522637 ,  0.8495707 , -0.48905239,  1.0652207 , -0.3079095 ,
        0.30141208,  0.02429047,  1.0293454 ,  1.1065664 , -0.13563903,
       -0.11545537,  0.37891278,  0.7820947 ,  0.35851318, -1.19

In [227]:
# total numberof extracted words.
vocab_len=len(w2v_model.wv)
print("The total number of words are : ", vocab_len)

The total number of words are :  29402


In [228]:
# print(vocab)
vocab=w2v_model.wv

In [229]:
# words most similar to a given word.
w2v_model.wv.most_similar('great')

[('good', 0.7815357446670532),
 ('fantastic', 0.760620653629303),
 ('wonderful', 0.7012671232223511),
 ('awesome', 0.6933570504188538),
 ('amazing', 0.6709727644920349),
 ('nice', 0.6701613664627075),
 ('excellent', 0.6624512672424316),
 ('decent', 0.626084566116333),
 ('terrific', 0.6160770058631897),
 ('terrible', 0.6078154444694519)]

In [230]:
# similaraity b/w two words
w2v_model.wv.similarity('good','great')

0.7815358

In [231]:
# cleaning reviews.
df['clean_review']=data['review_body'].apply(clean_reviews)

In [232]:
# number of unique words = 56379.

# now since we will have to pad we need to find the maximum lenght of any document.

maxi=-1
for i,rev in enumerate(df['clean_review']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

1720


In [233]:
tok = Tokenizer()
tok.fit_on_texts(df['clean_review'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['clean_review'])

In [234]:
max_rev_len=1720  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=300 # embedding dimension as choosen in word2vec constructor

In [235]:
# now padding to have a amximum length of 1565
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.

(100000, 1720)

In [236]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
    embed_vector=vocab.get_vector(word)
    if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero

In [237]:
# checking.
print(embed_matrix[10])

[ 8.99682865e-02 -1.05259621e+00 -6.29021287e-01 -5.59548795e-01
  1.19127190e+00  8.99894059e-01  5.09357274e-01 -7.48626471e-01
 -4.90565687e-01 -7.85346925e-01 -3.14486057e-01 -4.83407408e-01
 -1.14927195e-01  4.14361179e-01  9.12559748e-01  4.57611047e-02
 -4.11298841e-01 -1.64394641e+00 -4.66870606e-01  5.99127233e-01
 -2.94571612e-02  7.45729148e-01 -1.40267849e+00 -5.59033990e-01
 -9.69717383e-01  6.62935793e-01 -2.89381051e+00 -3.99250239e-02
 -7.27586687e-01 -6.72767997e-01  5.28688729e-01 -9.94669348e-02
  1.29635796e-01 -1.05314624e+00 -1.13817680e+00 -1.31791365e+00
 -1.64992487e+00 -1.16549827e-01  4.28857595e-01  1.22088015e+00
 -9.73775923e-01  8.10942471e-01  1.16047315e-01  2.17652410e-01
 -1.34454262e+00 -1.07243143e-01 -8.78000259e-01 -2.34663531e-01
  1.00955176e+00  3.18399698e-01  5.30542195e-01 -1.79542625e+00
  6.23048663e-01 -6.01855636e-01  5.40060282e-01  1.46226808e-01
  5.37211001e-01  2.14100733e-01 -4.95496958e-01  3.91541213e-01
 -2.71837227e-02 -1.57672

In [238]:
# prepare train and val sets first
#Y=keras.utils.to_categorical(df['sentiment'])  # one hot target as required by NN.
x_train,x_test,y_train,y_test=train_test_split(pad_rev,data['sentiment'],test_size=0.10,random_state=42)

In [240]:
#y_train = np.asarray(train_labels).astype('float32').reshape((-1,1))
#y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

# LSTM MODEL GENERATION  

In [252]:
layers = 128
dropout = 0.2
epochs = 3
batch_size = 128
maxlen=1720

In [253]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 128, input_length=maxlen))
model.add(tf.keras.layers.LSTM(layers))
model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1720, 128)         35865216  
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 35,996,929
Trainable params: 35,996,929
Non-trainable params: 0
_________________________________________________________________


In [256]:
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']



In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc',color="red")
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss',color="red")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
score = model.evaluate(X_test, y_test)
score[1]